In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import sys
import argparse
import subprocess
import pdb
import time
import random
import _pickle as cPickle
import matplotlib.pyplot as plt
import glob

%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf

from data_structure import get_batches
from hntm import HierarchicalNeuralTopicModel
from tree import get_descendant_idxs
from evaluation import validate, get_hierarchical_affinity, get_topic_specialization, print_topic_sample
from configure import get_config

# load data & set config

In [3]:
config = get_config(nb_name)

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu
np.random.seed(config.seed)
random.seed(config.seed)

In [5]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)
config.dim_bow = len(bow_idxs)

In [6]:
def debug_shape(variables, model):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, model, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    return _variables

# run

## initialize log

In [7]:
checkpoint = []
losses_train = []
ppls_train = []
ppl_min = np.inf
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','','','','','VALID:','','','','','TEST:','', 'SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','LOSS','PPL','NLL','KL','REG','LOSS','PPL', '1', '2', '3', 'CHILD', 'OTHER']]))))

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

def update_checkpoint(config, checkpoint, global_step):
    checkpoint.append(config.path_model + '-%i' % global_step)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0) + '.*'
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize model

In [8]:
if 'sess' in globals(): sess.close()
model = HierarchicalNeuralTopicModel(config)
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=config.max_to_keep)
update_tree_flg = False

## train & validate model

In [10]:
time_start = time.time()
while epoch < config.n_epochs:
    # train
    for ct, batch in train_batches:
        feed_dict = model.get_feed_dict(batch)
        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch, global_step_log = \
        sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_reg, model.topic_ppls, tf.train.get_global_step()], feed_dict = feed_dict)

        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

        if global_step_log % config.log_period == 0:
            # validate
            loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)

            # test
            if ppl_dev < ppl_min:
                ppl_min = ppl_dev
                loss_test, _, _, _, ppl_test, _ = validate(sess, test_batches, model)
                saver.save(sess, config.path_model, global_step=global_step_log)
                cPickle.dump(config, open(config.path_config % global_step_log, 'wb'))
                update_checkpoint(config, checkpoint, global_step_log)
            
            # visualize topic
            topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
            topics_freq_idxs = bow_idxs[topics_freq_indices]
            topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
            topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
            descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
            recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
            
            depth_specs = get_topic_specialization(sess, model, instances_test)
            hierarchical_affinities = get_hierarchical_affinity(sess, model)
            
            # log
            clear_output()
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev, \
                    '%.2f'%loss_test, '%.0f'%ppl_test, \
                    '%.2f'%depth_specs[1], '%.2f'%depth_specs[2], '%.2f'%depth_specs[3], \
                    '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            display(log_df)
            cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
            print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

            # update tree
            if not config.static:
                config.tree_idxs, update_tree_flg = model.update_tree(topic_prob_topic, recur_prob_topic)
                if update_tree_flg:
                    print(config.tree_idxs)
                    name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
                    if 'sess' in globals(): sess.close()
                    model = HierarchicalNeuralTopicModel(config)
                    sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
                    name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
                    sess.run([name_tensors[name].assign(variable) for name, variable in name_variables.items()]) # restore parameters
                    saver = tf.train.Saver(max_to_keep=1)
                
            time_start = time.time()

    train_batches = get_batches(instances_train, config.batch_size, iterator=True)
    epoch += 1

loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)
topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
topics_freq_idxs = bow_idxs[topics_freq_indices]
topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
display(log_df)
print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

TRAIN:                           VALID:               \
       Time   Ep   Ct    LOSS  PPL     NLL    KL   REG    LOSS  PPL     NLL   
5000     53   10    9  111.92  510  111.18  0.73  0.01  104.18  476  103.19   
10000    47   20   19  111.40  488  110.41  0.99  0.01  103.85  460  102.65   
15000    48   30   29  111.15  477  110.02  1.13  0.00  103.81  455  102.49   
20000    48   40   39  111.00  470  109.78  1.22  0.00  103.76  451  102.36   
25000    46   50   49  110.89  465  109.60  1.29  0.00  103.65  446  102.20   
30000    48   60   59  110.81  461  109.45  1.35  0.00  103.64  444  102.12   
35000    50   70   69  110.74  458  109.34  1.40  0.00  103.62  442  102.06   
40000    51   80   79  110.68  456  109.24  1.45  0.00  103.54  439  101.94   
45000    57   90   89  110.65  453  109.16  1.48  0.01  103.55  435  101.84   
50000    62  100   99  110.61  451  109.08  1.51  0.02  103.46  433  101.74   
55000    57  110  109  110.58  450  109.01  1.54  0.02  103.40  431  101.68   
60000    61  120  119  110.54  448  108.95  1.57  0.02  103.39  429  101.61   
65000    61  130  129  110.51  447  108.90  1.59  0.02  103.35  427  101.57   
70000    59  140  139  110.49  445  108.85  1.62  0.02  103.35  428  101.54   
75000    56  150  149  110.46  444  108.80  1.64  0.02  103.34  426  101.52   
80000    59  160  159  110.44  443  108.75  1.66  0.02  103.31  426  101.48   
85000    56  170  169  110.42  442  108.71  1.68  0.02  103.30  425  101.45   
90000    60  180  179  110.39  441  108.67  1.70  0.02  103.28  425  101.40   
95000    52  190  189  110.38  440  108.64  1.71  0.02  103.40  428  101.54   
100000   61  200  199  110.36  439  108.60  1.73  0.02  103.31  425  101.39   
105000   51  210  209  110.34  438  108.57  1.75  0.02  103.28  424  101.38   
110000   53  220  219  110.32  437  108.54  1.76  0.02  103.25  423  101.31   
115000   52  230  229  110.31  437  108.51  1.78  0.02  103.24  422  101.30   
120000   60  240  239  110.29  436  108.48  1.79  0.02  103.24  421  101.24   
125000   53  250  249  110.28  435  108.45  1.81  0.02  103.19  420  101.21   
130000   69  260  259  110.26  435  108.43  1.82  0.02  103.18  418  101.17   
135000   70  270  269  110.25  434  108.40  1.83  0.02  103.18  418  101.17   
140000   75  280  279  110.24  433  108.38  1.84  0.02  103.15  417  101.11   
145000   69  290  289  110.22  433  108.35  1.86  0.02  103.17  417  101.12   
150000   66  300  299  110.21  432  108.33  1.87  0.02  103.15  417  101.08   
...     ...  ...  ...     ...  ...     ...   ...   ...     ...  ...     ...   
350000   63  701  200  109.95  419  107.82  2.13  0.01  103.16  410  100.89   
355000   56  711  210  109.94  419  107.81  2.14  0.01  103.14  410  100.89   
360000   60  721  220  109.94  419  107.80  2.14  0.01  103.15  410  100.89   
365000   57  731  230  109.94  418  107.79  2.14  0.01  103.14  410  100.89   
370000   63  741  240  109.93  418  107.79  2.15  0.01  103.14  410  100.86   
375000   59  751  250  109.93  418  107.78  2.15  0.01  103.12  409  100.86   
380000   62  761  260  109.93  418  107.77  2.15  0.01  103.11  408  100.82   
385000   61  771  270  109.92  418  107.76  2.16  0.01  103.10  409  100.85   
390000   68  781  280  109.92  418  107.76  2.16  0.01  103.08  408  100.80   
395000   60  791  290  109.92  417  107.75  2.16  0.01  103.09  408  100.83   
400000   61  801  300  109.91  417  107.74  2.17  0.01  103.10  409  100.82   
405000   62  811  310  109.91  417  107.74  2.17  0.01  103.12  409  100.86   
410000   64  821  320  109.91  417  107.73  2.17  0.01  103.11  409  100.83   
415000   59  831  330  109.90  417  107.73  2.18  0.01  103.13  410  100.86   
420000   64  841  340  109.90  417  107.72  2.18  0.01  103.12  409  100.83   
425000   60  851  350  109.90  416  107.71  2.18  0.01  103.14  410  100.86   
430000   63  861  360  109.89  416  107.71  2.19  0.01  103.14  409  100.84   
435000   64  871  370  109.89  416  107.70  2.19  0.01  103.13  410  100.

0 R: 1.000 P: 0.406 quality - nice price bought 'm made ... time 've
   1 R: 0.273 P: 0.134 pocket room ipad small mouse charger power inside carry netbook
     11 R: 0.108 P: 0.108 carry pockets room shoulder compartments compartment back comfortable plenty straps
     13 R: 0.030 P: 0.030 ; & tablet pro air perfectly size big mouse ipad
   3 R: 0.133 P: 0.063 ! love perfectly mac perfect recommend pro color chromebook highly
     32 R: 0.038 P: 0.038 ! love bought gift perfect school recommend loves quality college
     31 R: 0.032 P: 0.032 ! love color buy recommend perfectly price perfect fast bought
   2 R: 0.070 P: 0.039 sleeve air smell pro protection inch neoprene retina inside protect
     22 R: 0.031 P: 0.031 item return received shipping ordered arrived seller amazon service customer
   5 R: 0.119 P: 0.074 bottom cover top scratches hard air plastic speck mac shell
     51 R: 0.045 P: 0.045 cover color keyboard screen protector blue picture love perfectly apple
